In [3]:
import MDAnalysis
from clustercode import ClusterEnsemble
import matplotlib 
import matplotlib.pyplot as plt

In [7]:
'''
This is a small atomistic trajectory including a single large micelle of aggregation number 100.
The Micelle is composed of SDS which has a HSO4 headgroup and carbon tail groups 
'''
xtc = "files/traj_small.xtc"
tpr = "files/topol_small.tpr"

# First we are going to look at the trajectory with some basic MDAnalysis commands
universe = MDAnalysis.Universe(tpr, xtc)

atom_names = set(universe.atoms.names)
residue_names = set(universe.residues.resnames)
atom_number = universe.atoms.n_atoms
residue_number = universe.residues.n_residues
print('Residue names: {:s}'.format(', '.join(residue_names)))
print('Number of residiue: {:d}'.format(residue_number))
print('Atom names: {:s}'.format(', '.join(atom_names)))
print('Number of atoms: {:d}'.format(atom_number))
print('Length of trajectory: {:d}'.format(universe.trajectory.n_frames))

Residue names: SOL, Na, SDS, Cl
Number of residiue: 6130
Atom names: C11, C4, Na, C1, C3, C8, OS4, C12, OS1, C6, Cl, C9, S, C2, C10, HW1, C7, OS2, C5, HW2, OW, OS3
Number of atoms: 19174
Length of trajectory: 71


In [8]:
'''
We want two molecules to be considered in a cluster when their carbon tails are close enough together. 
Therefore the 'cluster_objects' will be C1 - C12
We initialise the ClusterEnsemble object as follows:
'''
cluster_objects = ['C{:d}'.format(i) for i in range(1,13)]

ClstrEns = ClusterEnsemble(tpr, xtc, cluster_objects)

In [9]:
'''
Run the analysis for each frame within the specified times. cut-off is in Angstroem and describes how close two
objects need to be to considered in the same cluster. This still depends on the measure parameter which is either
centre of geometry (COG), centre of mass (COM) or bead to bead (b2b). 
'''
ClstrEns.cluster_analysis(cut_off=7.5, times=(60e3, 70e3), measure="COM", algorithm="dynamic", work_in="Residue", style="atom")

****TIME: 60000.00
---->Number of clusters 1
****TIME: 61000.00
---->Number of clusters 1
****TIME: 62000.00
---->Number of clusters 1
****TIME: 63000.00
---->Number of clusters 1
****TIME: 64000.00
---->Number of clusters 1
****TIME: 65000.00
---->Number of clusters 1
****TIME: 66000.00
---->Number of clusters 1
****TIME: 67000.00
---->Number of clusters 1
****TIME: 68000.00
---->Number of clusters 1
****TIME: 69000.00
---->Number of clusters 1
****TIME: 70000.00
---->Number of clusters 1


In [10]:
# This trajectory is larger and describes the inital phase of micellisations
xtc = "files/traj_large.xtc"
tpr = "files/topol_large.tpr"
ClstrEns = ClusterEnsemble(tpr, xtc, cluster_objects)

In [11]:
ClstrEns.cluster_analysis(cut_off=3.5, times=None, measure="COM", algorithm="dynamic", work_in="Residue", style="atom")

****TIME:     0.00
---->Number of clusters 9
****TIME:    50.00
---->Number of clusters 9
****TIME:   100.00
---->Number of clusters 6
****TIME:   150.00
---->Number of clusters 6
****TIME:   200.00
---->Number of clusters 4
****TIME:   250.00
---->Number of clusters 10
****TIME:   300.00
---->Number of clusters 6
****TIME:   350.00
---->Number of clusters 5
****TIME:   400.00
---->Number of clusters 9
****TIME:   450.00
---->Number of clusters 5
****TIME:   500.00
---->Number of clusters 3
****TIME:   550.00
---->Number of clusters 3
****TIME:   600.00
---->Number of clusters 3
****TIME:   650.00
---->Number of clusters 4
****TIME:   700.00
---->Number of clusters 3
****TIME:   750.00
---->Number of clusters 3
****TIME:   800.00
---->Number of clusters 8
****TIME:   850.00
---->Number of clusters 5
****TIME:   900.00
---->Number of clusters 4
****TIME:   950.00
---->Number of clusters 5
****TIME:  1000.00
---->Number of clusters 7
****TIME:  1050.00
---->Number of clusters 5
****TIME:

In [12]:
"""
These function will be added to the object soond as of now they are 
here to use.
"""
def get_cluster_size(cluster_list, frame):
    '''
    This function calculates the size of cluster in a single frame
    '''
    return [len(cluster) for cluster in cluster_list[frame]]

def get_cluster_sizes(cluster_list, first=None, last=None, stride=1):
    '''
    This function calculates the sizes of clusters in multiple frames.
    start and stop are measured in frames.
    '''
    if first is None: first = 0
    if last is None: last = len(cluster_list)
    cluster_sizes = []
    frames = (first, last, stride)
    for frame in range(first, last, stride):
        cluster_sizes.append(_get_cluster_size(cluster_list, frame))
    return cluster_sizes


def plot_histogram(ax, cluster_list, frames=[(0, 1, 1)], density=True, maxbins=True, *args, **kwargs):
    """
    Function to plot histograms for different timeframes.
    Frames is a list of tuples with (first, last, stride) for get_cluster_sizes
    """ 
    masterlist = []
    if isinstance(frames, list): pass
    else: frames = [frames]

    for frame in frames:
        frame_lists = get_cluster_sizes(
            cluster_list, 
            first=frame[0], 
            last=frame[1], 
            stride=frame[2]
            )
        flat_list = [item for sublist in frame_lists for item in sublist]
        masterlist.append(flat_list)
            #@Todo if frame outside of scope, give warning 
    if maxbins == True:
        bins = max([max(item) for item in masterlist])
    else:
        bins = None
    for i, (item, frame) in enumerate(zip(masterlist, frames)):
        label = 'Frames: {:d} - {:d}'.format(frame[0], frame[1])
        _, bins, _ = ax.hist(item, bins=bins, density=density, label=label, *args, **kwargs, alpha=0.5)

In [13]:
cluster_list = ClstrEns.cluster_list
# Get the size distribution of clusters in the first frame:
frame_0_clusters = get_cluster_size(cluster_list, 0)
def print_clusters(frames, i):
    print('Cluter sizes in {:d}. frame: {:s}'.format(i, ', '.join([str(item) for item in frames])))
print_clusters(frame_0_clusters, 0)

# Get the size distribution of several frames (from 50 to 54 here). 
# The return value of _get_cluster_size and _get_cluster_sizes is actually 
# a list and list of list respectively
frames_clusters = get_cluster_sizes(cluster_list, first=50, last=55, stride=1)
for frame, i in zip(frames_clusters, range(50, 55)):
    print_clusters(frame, i)

Cluter sizes in 0. frame: 392, 1, 1, 1, 1, 1, 1, 1, 1


NameError: name '_get_cluster_size' is not defined

In [ ]:
fig, ax = plt.subplots()
plot_histogram(ax, cluster_list, frames=[(50, 70, 1), (250, 270, 1)], density=True, maxbins=False, rwidth=None)
ax.set_xlabel('Number of molecules in cluster')
ax.set_ylabel('Probability of cluster')
ax.legend()

In [ ]:
'''
In the cluster_list we have all a list of molecules in each cluter.
We can therefore calculate all kinds of things with it.
'''
n_frame = 150
n_cluster = 1
special_cluster = cluster_list[n_frame][n_cluster]

# A cluster is a Residuegroup, we can use all the methods defined for
# Residuegroups
print('Type of special_cluster: {:s}'.format(type(special_cluster).__name__))

COM = special_cluster.center_of_mass()
print('Centre of mass: {:.2f}, {:.2f}, {:.2f}'.format(*COM))
print('Centre of geomety: {:.2f}, {:.2f}, {:.2f}'.format(*special_cluster.center_of_geometry()))
print('Radius of gyration: {:.3f}'.format(special_cluster.radius_of_gyration()))
print('Ids of residues(molecules) in this cluster: ')
print(special_cluster.resids)
print('Type of residues(molecules) in this cluster: ')
print(special_cluster.resnames)